In [49]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import pandas as pd
import numpy as np
import math

In [8]:
avito_ad = pd.read_csv("val.csv")

avito_ad.isnull().values.any()

avito_ad.shape

(16237, 9)

In [9]:
avito_ad.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0


In [10]:
ad = avito_ad[['description', 'is_bad']]
ad.head()

,description,is_bad
0,Звонить 89425546881,1
1,"Мобильная перегородка, предназначена для разгр...",0
2,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,1
3,"Продаю кровать-трансформер производства ""Столп...",0
4,"Мот в отличном состоянии для своих лет, Родной...",0


In [24]:
import regex as re
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^0-9a-zA-Zа-яA-Я+]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[^0-9a-zA-Zа-яA-Я+]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [25]:
ads = []
sentences = list(ad['description'])
for sen in sentences:
    ads.append(preprocess_text(sen))

In [14]:
y = ad['is_bad']

In [28]:
ads[10]

'toyo observe g5 ice без грыж и порезов звонить на 89155903052 '

In [29]:
y[10]

1

In [30]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)


In [32]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [33]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [36]:
def tokenize_ads(text_description):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_description))

In [38]:
tokenized_ads = [tokenize_ads(token_ad) for token_ad in ads]

In [40]:
ads_with_len = [[token_ad, y[i], len(token_ad)]
                 for i, token_ad in enumerate(tokenized_ads)]


In [42]:
ads_with_len.sort(key=lambda x: x[2])

In [43]:
sorted_ads_labels = [(ads_lab[0], ads_lab[1]) for ads_lab in ads_with_len]


In [44]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_ads_labels, output_types=(tf.int32, tf.int32))

In [46]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [47]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 9), dtype=int32, numpy=
 array([[ 1040,  2361,     0,     0,     0,     0,     0,     0,     0],
        [ 3515, 21472, 27009,     0,     0,     0,     0,     0,     0],
        [ 1192, 19259, 14150, 15290,     0,     0,     0,     0,     0],
        [ 1044,  2232,  4246,  2290,     0,     0,     0,     0,     0],
        [ 6227, 12740, 12740, 19841,  8889,     0,     0,     0,     0],
        [ 6486, 23632, 16068, 17465,  2575, 21057,     0,     0,     0],
        [ 5986,  2575,  2581, 16086, 12740, 22394,     0,     0,     0],
        [ 1195, 10260, 29740, 14150, 29752, 15414,     0,     0,     0],
        [ 6353, 16932,  6353, 17134,  6353, 16068,     0,     0,     0],
        [ 6486,  2692, 21057, 20842, 24087, 17914,     0,     0,     0],
        [ 1187, 10260,  1182, 29747, 15290,  4278,     0,     0,     0],
        [ 1196, 16856, 14150, 29752, 18947, 14150,     0,     0,     0],
        [ 6486,  2575,  2620,  6227,  2509,  3486,  2538,     0,     0],
   

In [50]:
TOTAL_BATCHES = math.ceil(len(sorted_ads_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

# НАКОНЕЦ МОДЕЛЬ!)

In [51]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output


In [52]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5


In [53]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)


In [54]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])


In [55]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
458/458 [==============================] - 102s 221ms/step - loss: 0.2880 - accuracy: 0.8767
Epoch 2/5
458/458 [==============================] - 95s 208ms/step - loss: 0.1785 - accuracy: 0.9311
Epoch 3/5
458/458 [==============================] - 95s 206ms/step - loss: 0.1084 - accuracy: 0.9612
Epoch 4/5
458/458 [==============================] - 96s 209ms/step - loss: 0.0689 - accuracy: 0.9760
Epoch 5/5
458/458 [==============================] - 94s 205ms/step - loss: 0.0502 - accuracy: 0.9825


In [57]:
results = text_model.evaluate(test_data)
print(results)

50/50 [==============================] - 1s 9ms/step - loss: 0.3477 - accuracy: 0.8788
[0.34773239493370056, 0.8787500262260437]
